In [2]:
## IMPORTS
import random
from deap import base
from deap import creator
from deap import tools
import numpy as np
import timeit
start = timeit.default_timer()
from stage import fullPrint

In [3]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


In [4]:
def generate_random_num_attr():
  original = np.array([0,0,0,0,0,0,0,0])
  noises = []
  sum_list = []

  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0,3))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.2))

  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list
noises = []
noises.append(np.random.normal(0, 0.025))
noises.append(np.random.normal(0, 0.075))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.01))
def mutate(individual):
  size = len(individual)
  for i in range(0,size):
    if random.random() > 0.9:
      individual[i] = individual[i] + noises[i]
  return individual,

In [5]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [6]:
## change to our functinon
from stage import getStage
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
import random
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)
def evalOneMax(individual):
    
    print(individual[0])
    
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    for symbol in random.sample(allList,1000):
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    resultFile = open("resultML.txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(stockHolding/(transactionCopy.shape[0]))+"\n")
    resultFile.write(str(transactionCopy.iloc[-1]['total'])+"\n")
    resultFile.close()
    if stockHolding/(transactionCopy.shape[0])<8:
        return 0,
    return transactionCopy.iloc[-1]['total'],

In [7]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

In [8]:
def main():


    pop = toolbox.population(n=1024)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        
    # Begin the evolution
    for g in range(10):
        print("-- Generation %i --" % g)
        resultFile = open("resultML.txt","a")
        resultFile.write("-- Generation %i --" % g+"\n")
        resultFile.close()
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [9]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
main()

[0.768240679561228, 0.596538712268169, 0.2179188673833247, 1.007962019601664, 1.2813906870575575, 1.0232381849374268, 0.8976218272592134, 1.1947814409609714]
0.8447909284195606
770.4629897399768
[0.7186832004129194, 2.080176555686847, -0.036118437926885766, 1.0895796122668169, 1.1417064692172265, 1.1699921178624342, 0.9182754117549055, 0.801939737791793]
0.050318922749822824
75.42375149791543
[0.08446743275007085, 2.136249643924844, 0.3742287654028761, 1.0471632858755455, 1.0038388800692022, 1.106852784253625, 0.8172018850815483, 0.8377728043590822]
0.0
100
[0.4826644351987961, 1.8358622366273214, 0.19156278906023913, 1.0780792691103773, 1.0948487707870003, 1.0682419676171189, 0.8850401298657438, 0.8803760079906289]
0.026931254429482635
75.42375149791543
[0.9904371632285969, 1.8983996400712262, -0.07219036123009534, 1.064647016509399, 1.1554598722903475, 1.1984757185800565, 0.8370801557419817, 1.1219850907298432]
0.0
100
[0.4543280741402025, 1.1411175963957838, 0.07624802662950822, 1.0